### Utils

In [3]:
import base64
import os
import json
from datetime import datetime
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
import requests
import re
import json
from typing import Dict, Any

def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_latest_sprint_status(base_path):
    """
    Find the latest sprint directory and load the project-sprint-status.md file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    str: Content of the project-sprint-status.md file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return "No sprint directories found."
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-status.md in the latest sprint directory
        status_file = os.path.join(sprint_path, 'project-sprint-status.md')
        
        if os.path.exists(status_file):
            with open(status_file, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"project-sprint-status.md not found in {latest_sprint}."
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    
def load_latest_sprint_backlog(base_path):
    """
    Find the latest sprint directory and load the project-sprint-backlog.json file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    dict: Content of the project-sprint-backlog.json file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return {"error": "No sprint directories found."}
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-backlog.json in the latest sprint directory
        backlog_file = os.path.join(sprint_path, 'project-sprint-backlog.json')
        
        if os.path.exists(backlog_file):
            with open(backlog_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        else:
            return {"error": f"project-sprint-backlog.json not found in {latest_sprint}."}
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}
    
def export_transcript(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    transcript = state["transcript"]
    
    filename = state["meeting_type"].replace(" ","_") + str(id) + ".txt"
    
    os.makedirs(os.path.join(folder_path), exist_ok=True)
    
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the string to a text file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(transcript)

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    """
    Load data from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file.
    
    Returns:
    dict: A dictionary containing the loaded JSON data.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def render_mermaid_diagram(diagram_code: str) -> str:
    # Encode the Mermaid code
    encoded_diagram = base64.b64encode(diagram_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the original Mermaid code
        return f"```mermaid\n{diagram_code}\n```"
    
def format_mermaid(input_string):
    # Step 1: Remove redundant "```mermaid" at the start and end
    cleaned_string = input_string.replace('```mermaid', '').replace('```', '')
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = cleaned_string.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()

    return formatted_string	


def export_meeting_history(state, output_file='meeting_history.json'):
    """
    Export the meeting history to a JSON file.
    
    Args:
    state (dict): The state dictionary containing the meeting history.
    output_file (str): The name of the output file. Defaults to 'meeting_history.json'.
    
    Returns:
    None
    """
    meeting_history = state.get("meeting_history", [])
    
    # Ensure meeting_history is a list
    if not isinstance(meeting_history, list):
        meeting_history = [meeting_history]
    
    try:
        with open(output_file, 'w') as file:
            json.dump(meeting_history, file, indent=2)
        print(f"Meeting history exported successfully to {output_file}")
    except IOError:
        print(f"Error: Unable to write to file {output_file}")

    return state  # Return the state to maintain consistency with your workflow


def manage_sprint_folders(state, project_folder):
    """
    Manages sprint folders based on the meeting type.
    Creates a new sprint folder if necessary and adds required files.
    
    :param state: The current state dictionary
    :param project_folder: Path to the project folder
    :return: Updated state with new sprint information
    """
    if "planning" in state.get("meeting_type", "").lower() or "plan" in state.get("meeting_type", "").lower():
        # List all directories in the project folder
        directories = [d for d in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, d))]
        
        # Filter and find the highest sprint number
        sprint_numbers = [int(re.findall(r'\d+', d)[0]) for d in directories if d.startswith("sprint") and re.findall(r'\d+', d)]
        
        if sprint_numbers:
            new_sprint_number = max(sprint_numbers) + 1
        else:
            new_sprint_number = 1
        
        # Create new sprint folder
        new_sprint_folder = os.path.join(project_folder, f"sprint{new_sprint_number}")
        os.makedirs(new_sprint_folder, exist_ok=True)
        
        # Create project_sprint_status.md
        status_file_path = os.path.join(new_sprint_folder, "project_sprint_status.md")
        with open(status_file_path, 'w') as status_file:
            status_file.write(f"# Sprint {new_sprint_number} Status\n\nStatus details will be updated here.")
        
        # Create project_sprint_backlog.json
        backlog_file_path = os.path.join(new_sprint_folder, "project_sprint_backlog.json")
        initial_backlog = {}
        with open(backlog_file_path, 'w') as backlog_file:
            json.dump(initial_backlog, backlog_file, indent=2)
        
        # Update state with new sprint information
        state["current_sprint_number"] = new_sprint_number
        state["current_sprint_folder"] = new_sprint_folder
        state["sprint_status_file"] = status_file_path
        state["sprint_backlog_file"] = backlog_file_path
        
        print(f"Created new sprint folder: {new_sprint_folder}")
    else:
        print("Meeting type does not indicate a planning session. No new sprint folder created.")
    
    return state


def load_json(file_path: str) -> Dict[str, Any]:
    """
    Reads a JSON file and returns its contents as a dictionary.

    :param file_path: The path to the JSON file to be read
    :return: A dictionary containing the data from the JSON file
    :raises FileNotFoundError: If the specified file does not exist
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        print(f"Successfully loaded JSON from {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {str(e)}")
        raise


def get_latest_sprint_folder(project_folder: str) -> str:
    """
    Scans the project folder for sprint folders and returns the name of the latest sprint folder.

    :param project_folder: Path to the project folder (e.g., 'project1/')
    :return: Name of the latest sprint folder (e.g., 'sprint5'), or None if no sprint folders are found
    """
    # List all items in the project folder
    items = os.listdir(project_folder)

    # Filter for sprint folders and extract their numbers
    sprint_folders = []
    for item in items:
        if os.path.isdir(os.path.join(project_folder, item)):
            match = re.match(r'sprint(\d+)', item, re.IGNORECASE)
            if match:
                sprint_number = int(match.group(1))
                sprint_folders.append((item, sprint_number))

    # Sort sprint folders by number (descending) and return the latest
    if sprint_folders:
        latest_sprint = max(sprint_folders, key=lambda x: x[1])
        print(f"Latest sprint folder found: {latest_sprint[0]}")
        return latest_sprint[0]
    else:
        print("No sprint folders found.")
        return None

def load_recent_json(folder_path: str) -> Dict[str, Any]:
    """
    Finds the most recent JSON file in the specified folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the most recent JSON file
    :raises FileNotFoundError: If the specified folder does not exist or contains no JSON files
    :raises json.JSONDecodeError: If the most recent file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if not json_files:
            raise FileNotFoundError(f"No JSON files found in {folder_path}")
        
        # Find the most recent JSON file
        most_recent_file = max(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        file_path = os.path.join(folder_path, most_recent_file)
        
        # Read and parse the JSON file
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded most recent JSON from {file_path}")
        return data
    
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        raise

### Prompts


In [13]:
NOTE_TAKER_PROMPT = """
You are a Scrum master in the team.
You have meetings and to keep everything organized you make general notes after each meeting.This will help you and your team to work more efficiently.
However, it is essential to get this accurate. Every valuable piece of information that has been said needs to be included.
You don't want to oversimplify things - thus leave out valuable information. 
You just want to transform the transcript of the meeting to a readable format and leave out the recurring information. 
In this way, the note can be used as a valuable asset.

To get you started you will find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n\n <company data> \n {company_data} \n </company data> \n 
Here you can find information about the employees and their skilles and responsibilities. \n\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n 
Here you can find information about the current project that the team is working on, in general: \n\n <project general> \n {project_general} \n </project general> \n
Here you can find information about the project requirements : \n\n <project requirements> \n {project_requirements} \n </project requirements> \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n\n <project state> \n {project_state} \n </project state> \n  
Here you can find information about the state of the backlog: \n\n <project backlog> \n {project_backlog} \n </project backlog> \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n\n <meeting history> \n {meeting_history} \n </meeting history> \n

Here you will find the transcript of the meeting. Use this to make a note.
Transcript: \n\n <transcript> \n {transcript} \n </transcript> \n

IMPORTANT: 
Do not leave information out that might be valuable. 
Remember your task is not to make a summary thus oversimplify and reduce information by compressing it but to write out that has been said and transform the meeting transcript into readable format.
You should only response with the structured note.
When you are finished say FINISHED
"""


NOTE_APPROVAL_PROMPT = """
You are a Scrum master in the team.
You have meetings and to keep everything organized you make general notes after each meeting.This will help you and your team to work more efficiently.
However, it is essential to get this accurate. Every valuable piece of information that has been said needs to be included.
You have made the first draft from the meeting. 
Now you have to make sure that the note contain only what has been said, without adding anything that hasn't been said.

To get you started you will find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n\n <company data> \n {company_data} \n </company data> \n 
Here you can find information about the employees and their skilles and responsibilities. \n\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n 
Here you can find information about the current project that the team is working on, in general: \n\n <project general> \n {project_general} \n </project general> \n
Here you can find information about the project requirements : \n\n <project requirements> \n {project_requirements} \n </project requirements> \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n\n <project state> \n {project_state} \n </project state> \n  
Here you can find information about the state of the backlog: \n\n <project backlog> \n {project_backlog} \n </project backlog> \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n\n <meeting history> \n {meeting_history} \n </meeting history> \n

Here is your first draft: \n\n <note draft> \n {note_draft} \n </note draft> \n

Finally, here you will find the transcript of the meeting. Use this to verify the note doesnt contain false information.
Transcript: \n\n <transcript> \n {transcript} \n </transcript> \n

IMPORTANT: 
If you find anything in the notes that is false or was not mentioned during the meeting or in previous documents, respond with: NOTE CONTAINS FALSE INFO.
If everything in the notes is accurate and was mentioned during the meeting or in previous documents, respond with: NOTE CONTAINS VALID INFO
"""


REFLECTION_PROMPT = """
You are a Scrum master reviewing your own note-taking work.
Your task is to critically analyze the notes you've taken from the team meeting and ensure they accurately represent the discussion without adding or omitting important information.

To get you started you with this task find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n\n <company data> \n {company_data} \n </company data> \n 
Here you can find information about the employees and their skilles and responsibilities. \n\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n 
Here you can find information about the current project that the team is working on, in general: \n\n <project general> \n {project_general} \n </project general> \n
Here you can find information about the project requirements : \n\n <project requirements> \n {project_requirements} \n </project requirements> \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n\n <project state> \n {project_state} \n </project state> \n  
Here you can find information about the state of the backlog: \n\n <project backlog> \n {project_backlog} \n </project backlog> \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n\n <meeting history> \n {meeting_history} \n </meeting history> \n


Here is your first draft:

\n\n <note draft> \n {note_draft} \n </note draft> \n

Now, reflect on this draft and consider the following:

Accuracy: Have you included all valuable information from the transcript without adding anything that wasn't explicitly discussed in the meeting?
Completeness: Did you capture every important point mentioned in the meeting? Are there any parts of the discussion that you might have overlooked?
Clarity: Is the note in a readable format? Have you successfully transformed the transcript into a clear, organized document?
Relevance: Have you included only information relevant to the project and team? Did you successfully leave out recurring or unnecessary information?
Context: Did you use the provided company, project, and employee information appropriately to provide context without inserting it into the meeting notes as if it was discussed?
Structure: Is the note structured in a way that makes it easy for team members to find and use the information?
Objectivity: Have you maintained objectivity in your note-taking, avoiding personal interpretations or assumptions?
Action Items: Have you clearly identified all action items, assignments, and deadlines mentioned in the meeting?

Based on this reflection, identify any areas where your notes could be improved.
Be specific about what changes need to be made and why. Remember, your goal is to create an accurate, comprehensive, and useful record of the meeting without adding information that wasn't discussed or omitting valuable points.
If you find areas for improvement, please create a revised version of the notes addressing these issues. 
If you believe your original draft accurately represents the meeting discussion, explain why you think no changes are necessary.
IMPORTANT:
Respond only with the revised note, nothing else.
Your reflection and any revisions should focus solely on accurately representing the content of the meeting transcript.
For the dates and the names of the participants use the meeting history and the transcript. 
Note that in the transcript the participants are annotated with their name.
Do not add new information or interpretations that were not explicitly stated in the meeting.
When you have completed your reflection and any necessary revisions, please state "REFLECTION COMPLETED".
"""


### State

In [5]:
from typing import TypedDict
class NoteTakerState(TypedDict):
    company_data: str	
    project_general: str	
    project_requirements: str
    meeting_history: Dict[str, Any]
    employee_profiles: str	
    			
    project_state : str
    project_backlog : Dict[str, Any]

    meeting_type: str		
    transcript: str
    note_taker_halucinate : bool
    reflection_haluconate : bool
    note_draft : str
    note_after_reflection : str
    note_final : str
    
	

In [6]:
from abc import ABC, abstractmethod

class Graph(ABC):
    @abstractmethod
    def create_graph(self):
        pass
    
    @abstractmethod
    def run_graph(self, project_folder):
        pass

### Node functions

In [8]:
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens= 8192)



def note_taker_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_TAKER_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_state = state.get("project_state"),
      project_backlog = state.get("project_backlog"),

      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate a meeting note based on the provided information.")
	]
    response = model.invoke(messages)
    state["note_draft"] = response.content
    print(state["note_draft"])
    return state

def note_approvel_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_APPROVAL_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_state = state.get("project_state"),
      project_backlog = state.get("project_backlog"),

      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_draft")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Validate the node based on the provided information.")
	]
    response = model.invoke(messages)
    if "NOTE CONTAINS FALSE INFO" in response.content:
        state["note_taker_halucinate"] = True
        print(response.content)
    else:
        state["note_taker_halucinate"] = False
        print(response.content)
    return state



def reflection_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = REFLECTION_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_state = state.get("project_state"),
      project_backlog = state.get("project_backlog"),

      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_draft")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Make the second version of the draft based on the provided information.")
    ]
    response = model.invoke(messages)

    state["note_after_reflection"] = response.content
    print(state["note_after_reflection"])
    return state


def reflection_approvel_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_APPROVAL_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_state = state.get("project_state"),
      project_backlog = state.get("project_backlog"),

      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_after_reflection")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Validate the note based on the provided information.")
	]
    response = model.invoke(messages)
    if "NOTE CONTAINS FALSE INFO" in response.content:
        state["note_taker_halucinate"] = True
        print(response.content)
    else:
        state["note_taker_halucinate"] = False
        state["note_final"] = state["note_after_reflection"]
        print(response.content)
    return state	


### Graph


In [10]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class NoteTaker(Graph):
    def __init__(self):
        self.workflow = None

    def create_graph(self):
        workflow = StateGraph(NoteTakerState)

        # Define nodes
        workflow.add_node("note_taker", note_taker_node)
        workflow.add_node("note_approvel", note_approvel_node)
        workflow.add_node("reflection", reflection_node)
        workflow.add_node("reflection_approvel", reflection_approvel_node)
        
        # Define edges
        workflow.add_edge("note_taker", "note_approvel")
        workflow.add_conditional_edges(
            "note_approvel",
            lambda x: "reflection" if not x.get("note_taker_halucinate") else "note_taker",
            {
                "reflection": "reflection",
                "note_taker": "note_taker"
            }
        )
        workflow.add_edge("reflection", "reflection_approvel")
        workflow.add_conditional_edges(
            "reflection_approvel",
            lambda x: "reflection" if x.get("reflection_haluconate") else END,
            {
                "reflection": "reflection",
                END: END
            }
        )


        # Set entry point
        workflow.set_entry_point("note_taker")
        self.workflow = workflow.compile()
    
    def run_graph(self, project_folder):
        # TODO : make folder management and file handling
        if self.workflow is None:
            raise ValueError("Graph has not been created. Call create_graph() first.")
        generate_meeting_state = load_recent_json(folder_path=os.path.join(project_folder, "state-logs"))
    
        state = NoteTakerState(
            company_data=load_markdown_to_str(file_path=os.path.join(project_folder, "company-general.md")),
            project_general=load_markdown_to_str(file_path=os.path.join(project_folder, "project-general.md")),
            project_requirements=load_markdown_to_str(file_path= os.path.join(project_folder, "project-requirements.md")),
            employee_profiles=load_markdown_to_str(file_path= os.path.join(project_folder, "employee-profiles.md")),

            meeting_history=load_from_json(file_path= os.path.join(project_folder, "meeting-history.json")),
            
			project_state=load_markdown_to_str(file_path= os.path.join(project_folder, "project-state.md")),
            project_backlog=load_markdown_to_str(file_path= os.path.join(project_folder, "project-backlog.md")),

            meeting_type=generate_meeting_state.get("meeting_type"),
            transcript=generate_meeting_state.get("transcript"),
        
            note_taker_halucinate = False,
            reflection_haluconate = False,
            note_draft = "",
            note_after_reflection = "",
            note_final = ""
        )
        state = self.workflow.invoke(state)
        return state
        
    def export_files(self, state: NoteTakerState, project_folder: str):
        export_state(state=state, folder_path=os.path.join(project_folder, "state-logs"), filename="note_taker")


In [14]:
graph = NoteTaker()
graph.create_graph()
state = graph.run_graph(project_folder="../../../data_/project1/")
graph.export_files(state=state, project_folder="../../../data_/project1/")
state


Successfully loaded most recent JSON from ../../../data_/project1/state-logs/generate_transcript1.json
Meeting Note: Sprint Planning for HealthTrack Pro

Date: 2024-06-17
Type: Sprint Planning
Attendees: Sarah Chen (Project Manager/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA Engineer/DevOps Specialist), Liam Foster (UI/UX Designer)

1. Project Overview
   - HealthTrack Pro: Comprehensive web application for personal health management
   - Features: Track daily activities, nutrition, health metrics; provide insights and recommendations

2. Sprint Details
   - Duration: 2 weeks
   - Sprint Goal: Implement user authentication and basic profile management

3. MVP Features Prioritization
   - Core features for Phase 1:
     a. User authentication
     b. Basic profile management
     c. Activity tracking (steps and exercise)
     d. Basic nutrition logging
     e. Simple health metrics d

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [15]:

graph.export_files(state=state, project_folder="../../../data_/project1/")
